In [1]:
import os
import json
from collections import defaultdict

In [2]:
def load_simulation_results(filename="simulation_results.json"):
    try:
        with open(filename, "r") as f:
            data = json.load(f)
        # Convert string keys back to tuple keys
        converted_data = {}
        for key_str, value in data.items():
            # Split the key string by "-" to get the cards as tuple
            tuple_key = tuple(key_str.split("-"))
            converted_data[tuple_key] = value
        return converted_data
    except FileNotFoundError:
        return {}  # Return empty dict if file doesn’t exist

In [3]:
winners_dict_filepath="winner_4_player_results.json"
losers_dict_filepath="loser_4_player_results.json"
winners_dict = load_simulation_results(winners_dict_filepath)
losers_dict = load_simulation_results(losers_dict_filepath)

In [4]:
win_rates = {}

# Get all unique hands from both dicts
all_hands = set(winners_dict.keys()).union(losers_dict.keys())

In [5]:
for hand in all_hands:
    wins = winners_dict.get(hand, 0)
    losses = losers_dict.get(hand, 0)
    total = wins + losses
    if total > 0:
        rate = wins / total
    else:
        rate = 0  # or None if you prefer
    win_rates[hand] = rate

In [6]:
results_list = list(sorted(win_rates.items(), key=lambda item: item[1], reverse=True))

In [7]:
def split_rank_suit(card_str):
    # Rank is everything except last char, suit is last char
    # Assumes card_str like '10♠' or 'A♣' (suit is 1 char)
    rank = card_str[:-1]
    suit = card_str[-1]
    return rank, suit

# Containers for aggregation
pocket_pairs = defaultdict(list)     # key: rank, values: list of values
suited = defaultdict(list)            # key: tuple(sorted ranks), values: list of values
offsuit = defaultdict(list)           # key: tuple(sorted ranks), values: list of values

In [8]:
for (card1, card2), value in results_list:
    rank1, suit1 = split_rank_suit(card1)
    rank2, suit2 = split_rank_suit(card2)

    if rank1 == rank2:
        # Pocket pair category
        pocket_pairs[rank1].append(value)
    else:
        # Different rank
        ranks_sorted = tuple(sorted([rank1, rank2], key=lambda r: '23456789JQKA'.index(r[0].upper()) if r[0].upper() in '23456789JQKA' else 0))

        if suit1 == suit2:
            # suited
            suited[ranks_sorted].append(value)
        else:
            # offsuit
            offsuit[ranks_sorted].append(value)

# Optionally, reduce lists by averaging or summing
def average(lst):
    return sum(lst) / len(lst) if lst else 0

In [9]:
agg_pocket_pairs = {k: average(v) for k,v in pocket_pairs.items()}
agg_suited = {k: average(v) for k,v in suited.items()}
agg_offsuit = {k: average(v) for k,v in offsuit.items()}

RANK_ORDER = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

def rank_index(rank):
    """
    Return the index of the rank in RANK_ORDER.
    Handles '10' as a string as well.
    """
    try:
        return RANK_ORDER.index(rank)
    except ValueError:
        return -1  # or some default for unknown rank

# Modify the sorting key function to use rank_index instead of direct string indexing
ranks_sorted = tuple(sorted([rank1, rank2], key=lambda r: rank_index(r)))

# Sorting outputs should also use rank_index properly, for example:
for rank, avg_val in sorted(agg_pocket_pairs.items(), key=lambda x: rank_index(x[0])):
    print(f"{rank}{rank}: {avg_val:.4f}")

# Similarly for suited and offsuit categories:
def sort_keys_by_rank(keys):
    return sorted(keys, key=lambda x: (rank_index(x[0]), rank_index(x[1])))

print("\nSuited Hands Average Win Rates:")
for ranks in sort_keys_by_rank(agg_suited.keys()):
    avg_val = agg_suited[ranks]
    print(f"{ranks[0]}{ranks[1]} suited: {avg_val:.4f}")

print("\nOffsuit Hands Average Win Rates:")
for ranks in sort_keys_by_rank(agg_offsuit.keys()):
    avg_val = agg_offsuit[ranks]
    print(f"{ranks[0]}{ranks[1]} offsuit: {avg_val:.4f}")


22: 0.2192
33: 0.2368
44: 0.2632
55: 0.2849
66: 0.3115
77: 0.3435
88: 0.3717
99: 0.4034
1010: 0.4489
JJ: 0.4906
QQ: 0.5293
KK: 0.5793
AA: 0.6412

Suited Hands Average Win Rates:
23 suited: 0.1775
24 suited: 0.1814
25 suited: 0.1905
26 suited: 0.1828
27 suited: 0.1774
28 suited: 0.1836
29 suited: 0.1942
210 suited: 0.2084
2J suited: 0.2175
2Q suited: 0.2314
2K suited: 0.2506
2A suited: 0.2862
34 suited: 0.1978
35 suited: 0.2068
36 suited: 0.1937
37 suited: 0.1958
38 suited: 0.1906
39 suited: 0.1998
3J suited: 0.2249
3Q suited: 0.2418
3K suited: 0.2629
3A suited: 0.2979
45 suited: 0.2216
46 suited: 0.2144
47 suited: 0.2157
48 suited: 0.2088
49 suited: 0.2101
4J suited: 0.2376
4Q suited: 0.2475
4K suited: 0.2707
4A suited: 0.3093
56 suited: 0.2318
57 suited: 0.2341
58 suited: 0.2299
59 suited: 0.2304
5J suited: 0.2415
5Q suited: 0.2618
5K suited: 0.2843
5A suited: 0.3223
67 suited: 0.2471
68 suited: 0.2414
69 suited: 0.2434
6J suited: 0.2496
6Q suited: 0.2656
6K suited: 0.2879
6A suited: 

In [12]:
# Prepare a combined dict with string keys for uniformity
combined = {}

# Add pocket pairs with keys as repeated ranks like '22', '33' ...
for rank, win_rate in agg_pocket_pairs.items():
    combined[f"{rank}{rank}"] = round(win_rate,4)

# Add suited hands - format keys as '23 suited', '24 suited', etc.
for ranks, win_rate in agg_suited.items():
    key_str = f"{ranks[0]}{ranks[1]} suited"
    combined[key_str] = round(win_rate,4)

# Add offsuit hands - format keys as '23 offsuit', '24 offsuit', etc.
for ranks, win_rate in agg_offsuit.items():
    key_str = f"{ranks[0]}{ranks[1]} offsuit"
    combined[key_str] = round(win_rate,4)

# Now sort combined dictionary by win rate descending
sorted_combined = list(sorted(combined.items(), key=lambda item: item[1], reverse=True))

In [13]:
sorted_combined

[('AA', 0.6412),
 ('KK', 0.5793),
 ('QQ', 0.5293),
 ('JJ', 0.4906),
 ('1010', 0.4489),
 ('KA suited', 0.4149),
 ('99', 0.4034),
 ('QA suited', 0.3992),
 ('KA offsuit', 0.3909),
 ('JA suited', 0.389),
 ('QK suited', 0.3815),
 ('10A suited', 0.3781),
 ('QA offsuit', 0.3764),
 ('88', 0.3717),
 ('JK suited', 0.3689),
 ('JA offsuit', 0.3631),
 ('JQ suited', 0.3581),
 ('QK offsuit', 0.3555),
 ('10K suited', 0.355),
 ('10A offsuit', 0.3496),
 ('9A suited', 0.3484),
 ('10Q suited', 0.3448),
 ('77', 0.3435),
 ('JK offsuit', 0.3427),
 ('8A suited', 0.3414),
 ('7A suited', 0.3327),
 ('10J suited', 0.3326),
 ('10K offsuit', 0.3321),
 ('9K suited', 0.3287),
 ('JQ offsuit', 0.3274),
 ('9A offsuit', 0.324),
 ('5A suited', 0.3223),
 ('9Q suited', 0.3189),
 ('8A offsuit', 0.3164),
 ('9J suited', 0.3151),
 ('10Q offsuit', 0.314),
 ('6A suited', 0.3139),
 ('66', 0.3115),
 ('4A suited', 0.3093),
 ('8K suited', 0.3086),
 ('10J offsuit', 0.3062),
 ('109 suited', 0.3033),
 ('7A offsuit', 0.3029),
 ('9K offsu